In [128]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [129]:
data=pd.read_csv(r'/kaggle/input/ner-dataset/ner_datasetreference.csv',encoding = "ISO-8859-1")
len(data)

1048575

In [130]:
data=data.iloc[0:40000,:]

In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  1799 non-null   object
 1   Word        40000 non-null  object
 2   POS         40000 non-null  object
 3   Tag         40000 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [132]:
data.head(10)


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [133]:
index=data.dropna().index
index

Int64Index([    0,    24,    54,    68,    83,   108,   132,   153,   181,
              196,
            ...
            39826, 39844, 39861, 39870, 39886, 39901, 39922, 39931, 39956,
            39993],
           dtype='int64', length=1799)

In [134]:
sentence=[]
pos=[]
count=0
for idx,i in enumerate(index):
    one_Sentence=''
    one_pos=''
    if i ==index[-1]:
        break
    for i in range(index[idx],index[idx+1]):
        count+=1
        if count%20==0:
            data.iloc[i]['Word']='unk'
        one_Sentence+=' '+data.iloc[i]['Word']
        one_pos+=' '+data.iloc[i]['POS']
    sentence.append('sos'+one_Sentence.lower())
    pos.append('sos'+one_pos)

In [135]:
print(sentence[1])
print(pos[1])

sos families of soldiers killed in the conflict joined the protesters who carried banners with such unk as " bush number one terrorist " and " stop the bombings . "
sos NNS IN NNS VBN IN DT NN VBD DT NNS WP VBD NNS IN JJ NNS IN `` NNP NN CD NN `` CC `` VB DT NNS . ``


In [136]:
print(sentence[6])
print(pos[6])
data.iloc[index[6]:index[7]]

sos the london march came ahead of anti-war unk today in other cities , including rome , paris , and madrid .
sos DT NNP NN VBD RB IN JJ NNS NN IN JJ NNS , VBG NNP , NNP , CC NNP .


,Sentence #,Word,POS,Tag
132,Sentence: 7,The,DT,O
133,NaN,London,NNP,B-geo
134,NaN,march,NN,O
135,NaN,came,VBD,O
136,NaN,ahead,RB,O
137,NaN,of,IN,O
138,NaN,anti-war,JJ,O
139,NaN,unk,NNS,O
140,NaN,today,NN,O
141,NaN,in,IN,O


In [137]:
print('data size=',len(sentence))
data_size=len(sentence)
train_data=sentence[:int(data_size*.8)]
test_data=sentence[len(train_data):int(data_size*.9)]
val_data=sentence[int(data_size*.9):]
train_label=pos[:int(data_size*.8)]
test_label=pos[len(train_data):int(data_size*.9)]
val_label=pos[int(data_size*.9):]
print(len(train_data),len(test_data),len(val_data))
print(len(train_label),len(test_label),len(val_label))
print(train_data[0])
print(train_label[0])

data size= 1798
1438 180 180
1438 180 180
sos thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british unk from that country .
sos NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP CC VB DT NN IN JJ NNS IN DT NN .


In [138]:
transition_dict=defaultdict(int)
for sentence in train_label:
    sentence=sentence.split()
    for idx,pos in enumerate(sentence):
        if idx==len(sentence)-1:
            break
            
        transition_dict[(sentence[idx],sentence[idx+1])]=transition_dict[(sentence[idx],sentence[idx+1])]+1

In [139]:
transition_dict

defaultdict(int,
            {('sos', 'NNS'): 122,
             ('NNS', 'IN'): 726,
             ('IN', 'NNS'): 232,
             ('NNS', 'VBP'): 318,
             ('VBP', 'VBN'): 153,
             ('VBN', 'IN'): 351,
             ('IN', 'NNP'): 782,
             ('NNP', 'TO'): 69,
             ('TO', 'VB'): 439,
             ('VB', 'DT'): 173,
             ('DT', 'NN'): 1394,
             ('NN', 'IN'): 1327,
             ('NNP', 'CC'): 115,
             ('CC', 'VB'): 33,
             ('IN', 'JJ'): 346,
             ('JJ', 'NNS'): 623,
             ('IN', 'DT'): 1312,
             ('NN', '.'): 538,
             ('NNS', 'VBN'): 59,
             ('NN', 'VBD'): 272,
             ('VBD', 'DT'): 275,
             ('DT', 'NNS'): 256,
             ('NNS', 'WP'): 22,
             ('WP', 'VBD'): 31,
             ('VBD', 'NNS'): 41,
             ('IN', '``'): 10,
             ('``', 'NNP'): 11,
             ('NNP', 'NN'): 313,
             ('NN', 'CD'): 12,
             ('CD', 'NN'): 78,
       

In [140]:
def pos_to_number (label):
    vocab_pos={}
    for poss in label:
        poss=word_tokenize(poss)
        
        for pos in poss:
            if pos not in list(vocab_pos):
                vocab_pos[pos]=len(vocab_pos)
    return vocab_pos

In [141]:
vocab_pos=pos_to_number(train_label)
len(vocab_pos)

39

In [142]:
index=list(vocab_pos)
columns=index.copy()
columns.remove('sos')
transition_matrix=pd.DataFrame(data = np.zeros((len(index),len(columns))), 
                  index = index, 
                  columns = columns)
  

In [143]:
for i in transition_dict.keys():
    transition_matrix.loc[i[0],i[1]]=transition_dict[i]

In [144]:
alpha=0.001
transition_matrix=(transition_matrix+alpha)/((np.array(transition_matrix.sum(1)).reshape((-1,1)))+(len(columns)*alpha))

In [145]:
transition_matrix.sum(1)

sos     1.000001
NNS     1.000000
IN      1.000000
VBP     1.000000
VBN     1.000001
NNP     1.000000
TO      1.000001
VB      1.000001
DT      1.000000
NN      1.000000
CC      1.000001
JJ      1.000000
.       1.000000
VBD     1.000001
WP      1.000014
``      1.000000
CD      1.000000
PRP     1.000002
VBZ     1.000001
POS     1.000000
VBG     1.000002
RB      1.000002
,       1.000002
WRB     1.000012
$       1.000000
MD      1.000000
WDT     1.000000
JJR     1.000000
:       1.000000
JJS     1.000000
RP      1.000014
PDT     1.000198
NNPS    1.000000
EX      1.000000
RBS     1.000000
LRB     1.000000
RRB     1.000000
RBR     1.000000
;       1.000000
PRP$    0.999876
WP$     0.988150
dtype: float64

In [146]:
transition_matrix

,NNS,IN,VBP,VBN,NNP,TO,VB,DT,NN,CC,...,PDT,NNPS,EX,RBS,LRB,RRB,RBR,;,PRP$,WP$
sos,0.084839,0.059109,6.960873e-04,0.003478,0.216963,0.000696,6.953919e-07,0.327530,0.026426,0.013213,...,6.960873e-04,6.960873e-04,2.086871e-03,6.953919e-07,6.953919e-07,6.953919e-07,6.259223e-03,6.953919e-07,0.004173,NaN
NNS,0.009069,0.299254,1.310783e-01,0.024320,0.010305,0.038747,2.473580e-03,0.004947,0.011542,0.058944,...,4.121947e-07,4.121947e-07,4.121947e-07,4.121947e-07,1.236996e-03,4.121947e-07,2.061386e-03,8.248016e-04,NaN,0.000413
IN,0.061181,0.016614,2.637104e-07,0.001319,0.206222,0.001055,2.637104e-07,0.345988,0.088080,0.000264,...,2.639741e-04,1.582526e-03,5.276846e-04,2.637104e-07,2.637104e-07,2.637104e-07,2.637104e-07,2.637104e-07,0.030854,NaN
VBP,0.037878,0.064391,1.893803e-06,0.289754,0.032197,0.045453,1.895697e-03,0.164763,0.039772,0.000002,...,1.893803e-06,1.893803e-06,5.683303e-03,1.893803e-06,1.893803e-06,1.893803e-06,1.893803e-06,1.893803e-06,NaN,NaN
VBN,0.031809,0.348894,9.949922e-04,0.055665,0.048707,0.103377,9.939982e-07,0.097413,0.035785,0.018887,...,9.949922e-04,9.949922e-04,9.939982e-07,9.939982e-07,9.949922e-04,9.939982e-07,3.976987e-03,9.939982e-07,0.007953,NaN
NNP,0.036515,0.077986,1.304134e-02,0.000783,0.309856,0.017997,2.610824e-04,0.004695,0.081637,0.029995,...,2.608216e-07,1.251970e-02,2.608216e-07,2.608216e-07,1.826012e-03,3.390942e-03,7.827257e-04,2.608216e-07,0.000522,NaN
TO,0.021490,0.002867,1.432587e-06,0.001434,0.070198,0.000001,6.289070e-01,0.136097,0.031518,0.000001,...,1.432587e-06,1.434019e-03,1.432587e-06,1.432587e-06,1.432587e-06,1.432587e-06,1.432587e-06,1.432587e-06,0.005732,NaN
VB,0.084430,0.126644,1.319195e-06,0.069919,0.063323,0.032981,7.916490e-03,0.228222,0.072557,0.007916,...,1.319195e-06,1.319195e-06,1.319195e-06,1.319195e-06,1.319195e-06,1.319195e-06,3.958904e-03,1.319195e-06,0.046173,NaN
DT,0.082660,0.005489,6.461012e-04,0.006458,0.142072,0.000323,3.228892e-07,0.000646,0.450108,0.000323,...,3.228892e-07,1.614769e-03,3.228892e-07,1.614769e-03,3.228892e-07,3.228892e-07,3.232120e-04,3.228892e-07,NaN,NaN
NN,0.061556,0.299208,6.764542e-03,0.014882,0.032694,0.039008,2.254998e-03,0.005637,0.108455,0.039684,...,2.254772e-07,2.254772e-07,2.257027e-04,2.254772e-07,1.578566e-03,4.511799e-04,6.766571e-04,1.127612e-03,0.000226,0.000226


In [147]:
emisssion=defaultdict(int)
for sentence,label in zip(train_data,train_label):
    sentence=word_tokenize(sentence)
    label=label.split()

    for word,pos in zip(sentence,label):

        emisssion[(word,pos)]= emisssion[(word,pos)]+1

In [148]:
emisssion

defaultdict(int,
            {('sos', 'sos'): 1438,
             ('thousands', 'NNS'): 11,
             ('of', 'IN'): 810,
             ('demonstrators', 'NNS'): 5,
             ('have', 'VBP'): 139,
             ('marched', 'VBN'): 1,
             ('through', 'IN'): 18,
             ('london', 'NNP'): 12,
             ('to', 'TO'): 667,
             ('protest', 'VB'): 5,
             ('the', 'DT'): 1889,
             ('war', 'NN'): 27,
             ('in', 'IN'): 824,
             ('iraq', 'NNP'): 57,
             ('and', 'CC'): 567,
             ('demand', 'VB'): 2,
             ('withdrawal', 'NN'): 6,
             ('british', 'JJ'): 23,
             ('unk', 'NNS'): 122,
             ('from', 'IN'): 154,
             ('that', 'DT'): 17,
             ('country', 'NN'): 46,
             ('.', '.'): 1369,
             ('families', 'NNS'): 5,
             ('soldiers', 'NNS'): 33,
             ('killed', 'VBN'): 64,
             ('conflict', 'NN'): 8,
             ('joined', 'VBD'): 4,
  

In [149]:
def word_to_number (data):
    vocab={}
    for i in data:
        words=word_tokenize(i)
        for word in words:
            if word not in list(vocab):
                vocab[word]=len(vocab)
    return vocab

In [150]:
vocab_word=word_to_number(train_data)
len(vocab_word)

5288

In [151]:
columns=vocab_word.keys()
emisssion_matrix=pd.DataFrame(data = np.zeros((len(index),len(columns))), 
                  index = index, 
                  columns = columns)
  

In [152]:
emisssion_matrix.shape

(39, 5288)

In [153]:
for i in emisssion.keys():
    emisssion_matrix.loc[i[1],i[0]]=emisssion[i]

In [154]:
emisssion_matrix.shape

(41, 5288)

In [155]:
alpha=0.001
emisssion_matrix=(emisssion_matrix+alpha)/((np.array(emisssion_matrix.sum(1)).reshape((-1,1)))+(len(columns)*alpha))

In [156]:
emisssion_matrix

,sos,thousands,of,demonstrators,have,marched,through,london,to,protest,...,whoever,apparently,protecting,darien,occasional,incursions,hebron,shootout,planting,factional
sos,9.963368e-01,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,...,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07,6.928624e-07
NNS,4.113046e-07,4.524762e-03,4.113046e-07,2.056934e-03,4.113046e-07,4.113046e-07,4.113046e-07,4.113046e-07,4.113046e-07,4.113046e-07,...,4.113046e-07,4.113046e-07,4.113046e-07,4.113046e-07,4.113046e-07,4.117159e-04,4.113046e-07,4.113046e-07,4.113046e-07,4.113046e-07
IN,2.633458e-07,2.633458e-07,2.133104e-01,2.633458e-07,2.633458e-07,2.633458e-07,4.740488e-03,2.633458e-07,3.687105e-03,2.633458e-07,...,2.633458e-07,2.633458e-07,2.633458e-07,2.633458e-07,2.633458e-07,2.633458e-07,2.633458e-07,2.633458e-07,2.633458e-07,2.633458e-07
VBP,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,2.606490e-01,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,...,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06,1.875159e-06
VBN,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.898268e-04,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,...,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07,9.888380e-07
NNP,2.603971e-07,2.603971e-07,2.603971e-07,2.603971e-07,2.603971e-07,2.603971e-07,2.603971e-07,3.125026e-03,2.603971e-07,2.603971e-07,...,2.603971e-07,2.603971e-07,2.603971e-07,2.606575e-04,2.603971e-07,2.603971e-07,2.606575e-04,2.603971e-07,2.603971e-07,2.603971e-07
TO,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,9.484038e-01,1.421893e-06,...,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06,1.421893e-06
VB,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,2.620374e-02,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,6.551917e-03,...,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06,1.310121e-06
DT,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,...,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07,3.223427e-07
NN,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,6.758571e-04,...,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.252106e-07,2.254358e-04,2.252106e-07,2.252106e-07


In [157]:
print(len(word_tokenize(test_data[0])))
print(len(word_tokenize(test_label[0])))

26
26


In [158]:
index.remove('sos')

In [159]:
def prdiction(sample,index):
    label=['sos']
    sample=word_tokenize(sample)
    if sample[0]=='sos':
        pass
    else:
        sample=['sos']+sample

    for idx,word in enumerate(sample):
        if idx>=len(sample)-1:
            break
        try:
            emisssion_matrix.loc[:,sample[idx+1]]
        except:
            sample[idx+1]='unk'
        index_of_label=np.argmax(transition_matrix.loc[label[idx]]*emisssion_matrix.loc[:,sample[idx+1]][1:])
        label.append(index[index_of_label])
    return label

In [164]:
print(prdiction(test_data[2],index))
print(test_label[2].split())

['sos', 'DT', 'NN', 'MD', 'VB', 'VBN', 'IN', 'NNP', 'MD', 'VB', 'DT', 'TO', 'DT', 'NN', 'POS', 'NN', 'IN', ',', 'IN', 'DT', 'NN', 'TO', 'VB', 'VBN', 'IN', 'DT', 'NNS', '.']
['sos', 'DT', 'NN', 'MD', 'VB', 'VBN', 'IN', 'NNP', 'MD', 'VB', 'VBN', 'TO', 'DT', 'NN', 'POS', 'NNP', 'NNP', ',', 'IN', 'DT', 'NN', 'TO', 'VB', 'VBN', 'IN', 'JJ', 'NNS', '.']
